# SwellDB Table Chains
In this example we will explore how we can generate a chain of tables, using different operators, in order to generate the final table that contains all the information. Let's assume a use-case in which we would like to create a table containing resistance training exercises.

## SwellDB initialization

In [59]:
import os

import pyarrow as pa

# SwellDB imports
from swelldb import SwellDB, OpenAILLM
from swelldb.swelldb import Mode

# Initialize a SwellDB instance
swelldb: SwellDB = SwellDB(llm=OpenAILLM(model="gpt-4o"))

## Create the names table

In [38]:
names = (
    swelldb.table_builder()
    .set_table_name("groups")
    .set_content("Muscle groups")
    .set_schema("group_name str")
    .set_base_columns(["group_name"])
    .set_table_gen_mode(Mode.LLM)
).build()

## Create exercises table

In [39]:
exercises = (
    swelldb.table_builder()
    .set_table_name("question")
    .set_content("Resistance exercises per muscle group")
    .set_schema("exercise_name str, group_name str")
    .set_base_columns(["group_name"])
    .set_table_gen_mode(Mode.LLM)
).build()

## Create the YouTube links table

In [56]:
youtube_links = (
    swelldb.table_builder()
    .set_table_name("links")
    .set_content("YouTube links for the exercises")
    .set_schema("exercise_name str, link str")
    .set_base_columns(["exercise_name"])
    .set_table_gen_mode(Mode.SEARCH)
).build()

## Create a table chain

In [52]:
chain = (names | exercises | youtube_links)
chain.explain()

SearchEngineTable[schema=['exercise_name', 'link']
--LLMTable[schema=['exercise_name', 'group_name']
----LLMTable[schema=['group_name']


## Materialize the chain

In [60]:
data: pa.Table = chain.materialize()

In [61]:
data.to_pandas()

,exercise_name,link,group_name
0,Bench Press,https://www.youtube.com/watch?v=rT7DgCr-3pg,Chest
1,Push-Up,https://www.youtube.com/watch?v=IODxDxX7oi4&pp...,Chest
2,Pull-Up,https://www.youtube.com/watch?v=syS4M1G-rII&pp...,Lats
3,Pull-Up,https://www.youtube.com/watch?v=syS4M1G-rII&pp...,Back
4,Bent-Over Row,https://www.youtube.com/watch?v=QFq5jdwWwX4,Back
5,Shoulder Press,https://www.youtube.com/watch?v=B-aVuyhvLHU,Shoulders
6,Lateral Raise,https://www.youtube.com/watch?v=geenhiHju-o,Shoulders
7,Bicep Curl,https://www.youtube.com/watch?v=Nkl8WnH6tDU,Biceps
8,Hammer Curl,https://www.youtube.com/watch?v=zC3nLlEvin4,Biceps
9,Tricep Dip,https://www.youtube.com/watch?v=6kALZikXxLc,Triceps
